# Group Task 2 Answers

Given an RNA-Seq experiment with a knocked out gene, you were asked to answer the following questions:

  * **What is the name of the knock out gene?**
  
  
  * **What influence does it have?**   
  
  
  * **How did you determine those?**  
  

***

## Dataset

You were given the following files in order to conduct the analysis:

  * Wild type sample reads in FASTQ format  
    `WT[replicate]_[1|2].fq.gz`  


  * Knockout sample reads in FASTQ format  
    `KO[replicate]_[1|2].fq.gz`
  
  
  * _P. berghi_ genome in FASTA format  
    `PbANKA_v3.fasta`
    
    
  * _P. berghi_ transcripts in FASTA format  
    `Pb.CDS.fasta`
    

  * _P. berghi_ annotations in GFF format  
    `PbANKA_v3.gff3.gz`
    
    
  * _P. berghi_ gene descriptions in TSV format  
    `Pb.names.txt`
      
    
  * R script to run sleuth  
    `sleuth.R`

### Important questions

**Can you summarise the experimental design?**

The experimental design should explain what each sample represents, i.e. the conditions that were applied and how many replicates there were.  In this experiment, there are two conditions, wild type (WT) and knock out (KO), each of which has three biological replicates.

| Sample name | Condition | Replicate |
| :-: | :-: | :-: |
| WT | wild type | 1 |
| WT | wild type | 2 |
| WT | wild type | 3 |
| KO | knock out | 1 |
| KO | knock out | 2 |
| KO | knock out | 3 |

***

## Aligning sample reads to the genome

**First, we need to move into the directory containing the data.**

In [5]:
cd /home/manager/course_data/group_projects/RNASeq_2

**Then, we need to build our HISAT2 index for the genome.**

In [ ]:
hisat2-build PbANKA_v3.fasta PbANKA_v3_hisat2.idx

**Next, we can use a loop to align all of our sample files to the genome.**  

_Be patient, this will take a while!_

In [ ]:
for fname in *_1.fq.gz
do
    # Get sample name from file name
    sample=`echo "$fname" | cut -d'_' -f1`

    # Align sample to genome
    echo "Aligning sample..."${sample}
    hisat2 --max-intronlen 10000 -x PbANKA_v3_hisat2.idx \
    -1 ${sample}_1.fq.gz -2 ${sample}_2.fq.gz \
    -S ${sample}.sam

    # Convert SAM to sorted BAM
    echo "Converting sample SAM to sorted BAM..."${sample}
    samtools view -b ${sample}.sam | \
    samtools sort -o ${sample}.sorted.bam

    # Index sorted BAM
    echo "Indexing sample BAM..."${sample}
    samtools index ${sample}.sorted.bam
done

### Important questions

**What is the overall alignment rate of each of the samples?**

It is important to look at the overall alignment rate (for the genome) as this can give an idea of whether there are any issues with the experiment (e.g. contamination - like we saw in the practical).

| Sample name | Alignment rate|
| :-: | :-: |
| WT1 | % |
| WT2 | % |
| WT3 | % |
| KO1 | % |
| KO2 | % |
| KO3 | % |

This looks good, all of the samples have a relatively similar alignment rate >97%.

***

## Visualising the genome alignments 

Before you can use IGV to visualise the genome, you must first index the genome using `samtools faidx`.

**Index the genome with `samtools`.**

In [2]:
samtools faidx PbANKA_v3.fasta

Once that's finished, you need to load your genome (`PbANKA_v3.fasta`), annotation (`PbANKA_v3.gff3.gz`) and sorted alignment files (`[sample].sorted.bam`) into IGV.

**First, start IGV.**  

In [ ]:
igv.sh &

**Load the genome file `Genomes -> Load Genome from File`**  

**Load the annotation (gff) file `File -> Load from File`**   

**Load the sorted sample BAM files `File -> Load from File`** 

**Make sure to set the alignment tracks to "squished" and to view reads as "paired".**

**Type 'PBANKA_KO' in the search box and click 'Go'.**

This will give you a view like the one below.  Here, we have coloured the WT coverage plots blue and the KO coverage plots red to make it a little easier to see the difference.

![images/PBANKA_KO_IGV.png](images/PBANKA_KO_IGV.png)

### Important questions

**Where in the genome is PBANKA_KO located?**

You can get the co-ordinates of PBANKA_KO from the annotation file (`PbANKA_v3.gff3.gz`) using `grep` (first uncompressing the file with `gunzip`) or `zgrep`.

In [3]:
zgrep gene.*PBANKA_KO PbANKA_v3.gff3.gz

PbANKA_09_v3	chado	gene	196868	201157	.	+	.	ID=PBANKA_KO;Name=PBANKA_KO


PBANKA_KO is located on the **forward strand** of **PbANKA_09_v3** between **196868** and **201157**.

**How many exons does PBANKA_KO have?**

In IGV, you can see that PBANKA_KO has **two exons**.  

![images/PBANKA_KO_IGV_exon.png](images/PBANKA_KO_IGV_exon.png)

This can be confirmed by looking for the PBANKA_KO CDS annotations the GFF file. 

In [4]:
zgrep CDS.*PBANKA_KO PbANKA_v3.gff3.gz

PbANKA_09_v3	chado	CDS	196868	197309	.	+	0	ID=PBANKA_KO.1:exon:2;Parent=PBANKA_KO.1
PbANKA_09_v3	chado	CDS	197409	201157	.	+	2	ID=PBANKA_KO.1:exon:1;Parent=PBANKA_KO.1


**Were all of the exons in PBANKA_KO knocked out?**

No. Only the second exon was knocked out.

![images/PBANKA_KO_IGV_coverage.png](images/PBANKA_KO_IGV_coverage.png)

***

## Aligning sample reads to the transcriptome

Before we can use `kallisto` to align the sample reads to the transcriptome, we first need to build a kallisto index of the transcriptome using `kallisto index`.

**Build a Kallisto index of the tanscriptome (`Pb.CDS.fasta`) using `kallisto`.**

In [5]:
kallisto index -i Pb.CDS.kallisto Pb.CDS.fasta


[build] loading fasta file Pb.CDS.fasta
[build] k-mer length: 31
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 13763 contigs and contains 9949889 k-mers 



As with the genome alignments, we can run the transcriptome alignments for all the samples using a loop.

**Align your samples to the transcriptome using `kallisto quant`.**

In [6]:
for fname in *_1.fq.gz
do
    # Get sample name from file name
    sample=`echo "$fname" | cut -d'_' -f1`
    
    # Quantify transcript expression in sample
    echo "kallisto quantification for sample..."${sample}
    kallisto quant -i Pb.CDS.kallisto -o ${sample} -b 100 \
    ${sample}_1.fq.gz ${sample}_2.fq.gz
done

kallisto quantification for sample...KO1

[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 5,077
[index] number of k-mers: 9,949,889
[index] number of equivalence classes: 7,297
[quant] running in paired-end mode
[quant] will process pair 1: KO1_1.fq.gz
                             KO1_2.fq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 2,898,714 reads, 2,341,894 reads pseudoaligned
[quant] estimated average fragment length: 262.537
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 183 rounds
[bstrp] running EM for the bootstrap: 100

kallisto quantification for sample...KO2

[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 5,077
[index] number of k-mers: 9,949,889
[index] number of equivalence classes: 7,297
[quant] running in paired-end mode
[quant] will process pa

### Important questions

**How many transcripts are there?**

There are **5077** transcripts.

In [7]:
 grep -c '>' Pb.CDS.fasta

5077


***

## Run DE analysis in sleuth

To identify differentially expressed genes you can use the R package, `sleuth`.  

**Run the sleuth R script (`sleuth.R`).**

In [8]:
Rscript sleuth.R

Loading required package: methods
There were 15 warnings (use warnings() to see them)
Error in file(file, "rt") : cannot open the connection
Calls: read.table -> file
In addition: Warning message:
In file(file, "rt") :
  cannot open file 'hiseq_info.txt': No such file or directory
Execution halted


: 1

This should give you an error which contains:

    cannot open file 'hiseq_info.txt': No such file or directory
    
**So, let's take a look at the R script and see what's going on.**

In [9]:
cat sleuth.R

library("sleuth")
s2c <- read.table("hiseq_info.txt", header = TRUE, stringsAsFactors=FALSE)

sample_id <- c("WT1", "WT2","WT3", "KO1", "KO2", "KO3")
kal_dirs <- sapply(sample_id, function(id) file.path(".", id))
s2c <- dplyr::select(s2c, sample = sample, condition)
s2c <- dplyr::mutate(s2c, path = kal_dirs)

t2g<-read.table("Pb.names.txt", header=T, sep="\t")

so <- sleuth_prep(s2c, ~ condition, target_mapping = t2g, extra_bootstrap_summary = TRUE, read_bootstrap_tpm = TRUE)
so <- sleuth_fit(so)
so <- sleuth_fit(so, ~1, 'reduced')
so <- sleuth_wt(so, 'conditionWT', 'full')

results_table <- sleuth_results(so, 'conditionWT', test_type = 'wt')

write.table(results_table, file="kallisto.results", quote=FALSE, sep="\t", row.names=FALSE)

sleuth_live(so)


Look at the second line:

    s2c <- read.table("hiseq_info.txt", header = TRUE, stringsAsFactors=FALSE)
    
The script is looking for a file called `hiseq_info.txt`.  

**Let's see if the file has been given to you.**

In [10]:
ls hiseq_info.txt

ls: cannot access hiseq_info.txt: No such file or directory


: 2

Nope.  Well...we did warn you that some files might be missing! But, that still doesn't tell us what the `hiseq_info.txt` file contains...

**Let's take a look at the one we used in the practical.**

In [11]:
cat /home/manager/pathogen-informatics-training/Notebooks/RNA-Seq/data/hiseq_info.txt

sample	condition
MT1	MT
MT2	MT
SBP1	SBP
SBP2	SBP
SBP3	SBP


So, it looks like this file indicates which condition was applied to each of the samples.

**Copy the file from the practical into the same directory as your sleuth R script.**

In [12]:
cp /home/manager/pathogen-informatics-training/Notebooks/RNA-Seq/data/hiseq_info.txt .

**Let's check that worked.**

In [13]:
cat hiseq_info.txt

sample	condition
MT1	MT
MT2	MT
SBP1	SBP
SBP2	SBP
SBP3	SBP


Good, now let's update this file so it contains our sample names.

**You can edit the file manually by typing the following `nano` command in your terminal. Be careful which order you put the samples in.**

In [ ]:
nano /home/manager/course_data/group_projects/RNASeq_1/hiseq_info.txt

**Alternatively, you can make the edits using `sed`.**

In [14]:
sed -ie 's/MT/WT/g' hiseq_info.txt
sed -ie 's/SBP/KO/g' hiseq_info.txt
sed -ie '/^WT2/a WT3\tWT' hiseq_info.txt

**And, check that it's worked.**

In [15]:
cat hiseq_info.txt

sample	condition
WT1	WT
WT2	WT
WT3	WT
KO1	KO
KO2	KO
KO3	KO


Perfect, our six samples are now in the file.

**So, let's try running that R script again.**

In [ ]:
Rscript sleuth.R

Loading required package: methods
There were 15 warnings (use warnings() to see them)
reading in kallisto results
dropping unused factor levels
......
normalizing est_counts
4907 targets passed the filter
normalizing tpm
merging in metadata
summarizing bootstraps
......
fitting measurement error models
shrinkage estimation
6 NA values were found during variance shrinkage estimation due to mean observation values outside of the range used for the LOESS fit.
The LOESS fit will be repeated using exact computation of the fitted surface to extrapolate the missing values.
These are the target ids with NA values: PBANKA_0007601, PBANKA_0008001, PBANKA_0837201, PBANKA_1200021, PBANKA_1465861, PBANKA_1342500
computing variance of betas
fitting measurement error models
shrinkage estimation
2 NA values were found during variance shrinkage estimation due to mean observation values outside of the range used for the LOESS fit.
The LOESS fit will be repeated using exact computation of the fitted surf

**Click [http://127.0.0.1:42427](http://127.0.0.1:42427) to open the sleuth results in your web browser.**

### Important questions

**Can you summarise the data that's been processed (i.e. number of reads processed and the proportion of reads mapping to the genome and transcriptome)?**

You can get a summary of the processed data by going to `summaries -> processed data`. 

![images/kallisto_processed_data.png](images/kallisto_processed_data.png)

**Looking at the PCA plot (`maps -> PCA`) do you think the samples form tight, distinct clusters based on the condition (WT or KO) that was applied?**

Reasonably, yes.  There is a clear vertical split between the WT and the KO samples.

![images/sleuth_original_pca.png](images/sleuth_original_pca.png)

You can also look at the proportion of variance explained by each principal component (PC). As this is a single factor experiment, we would expect that if there were variation, most of this would be explained by the first principal component, PC1. Broadly speaking, this represents the variation resulting from the difference in condition (WT vs KO). You can see here that >90% of the variance is explained by PC1 (the vertical axis of the PCA plot above). 

![images/sleuth_original_pca_bar.png](images/sleuth_original_pca_bar.png)

If you were to rerun the analysis with KO3 removed, the PCA plot does become a little clearer.

![images/sleuth_processed_pca.png](images/sleuth_processed_pca.png)

**Was PBANKA_KO differentially expressed?**

**Yes** as it's significantly (q-value < 0.05) more highly expressed (b > 0) in the WT samples. The beta value may be lower than you expect as only one of the two exons was knocked out (i.e. the reads mapping to the first exon will be counted towards the PBANKA_KO gene expression in the KO samples).

For this, you need to go to `analyses -> test table` and enter PBANKA_KO in the search box.

![images/sleuth_test_table.png](images/sleuth_test_table.png)

You can also look at the expression profiles by going to `analyses -> transcript view` and typing PBANKA_KO in the search box.

![images/sleuth_transcript_view.png](images/sleuth_transcript_view.png)

**How many genes are more highly expressed in the WT samples than in the KO samples?**

**828**

In [17]:
awk -F'\t' '$4 < 0.01 && $5 > 0' kallisto.results | wc -l

828


**How many genes are more highly expressed in the KO samples than in the WT samples?**

**748**

In [18]:
awk -F'\t' '$4 < 0.01 && $5 < 0' kallisto.results | wc -l

748


**Write the gene IDs of the significantly differentially expressed genes to files for the next part of the analysis.**

In [19]:
awk -F'\t' '$4 < 0.01 && $5 > 0  {print $1}' kallisto.results > kallisto.WT.sig.genes
awk -F'\t' '$4 < 0.01 && $5 < 0  {print $1}' kallisto.results > kallisto.KO.sig.genes

***

## GO term enrichment analysis

Gene ontology (GO) terms are a dictionary which can be used to assign functions to a gene or transcript. You can use [http://www.plasmodb.org](http://www.plasmodb.org) to perform a GO term enrichment analysis (i.e. which terms are significantly more abundant in your differentially expressed genes than in all of the genes as a whole).

**Go to [http://www.plasmodb.org](http://www.plasmodb.org) in your web browser.**

**Go to `My Strategies -> New`.**

**Go to `Annotation, curation and identifiers -> Gene IDs`.**

**Upload your file of gene IDs that were more highly expressed in the WT samples.**

**Go to `Analyse results` (blue button) and `GO enrichment`.**

**You want to do a GO analysis using the molecular functions (MF).** 

### Important questions

**Which GO terms (molecular functions) are enriched in genes with higher expression in the WT samples?**

You can get this from the table that the analysis generates. You could say that broadly speaking that this gene is involved in the regulation of motility, adhesion and the cell cycle.

![images/WT_MF_table.png](images/WT_MF_table.png)

You can also use some of the other output options to find interesting ways of displaying this data.

![images/WT_MF_words.png](images/WT_MF_words.png)

**Which GO terms (molecular functions) are enriched in genes with higher expression in the KO samples?**

You'll need to run the same analysis with your KO file and look at the results table. It looks like there are changes in ribosomal processes.

![images/KO_MF_table.png](images/KO_MF_table.png)

And again, there are several useful ways to visualise your results.

![images/KO_MF_words.png](images/KO_MF_words.png)

***

## What is PBANKA_KO?

So, we've seen the influences of the knock out gene, but what is it? 

For this group task, we removed the real name of PBANKA_KO from all of the files we gave you. How mean! To get the real name of PBANKA_KO, we need the real genome annotation file.

**Download the real annotation file from the FTP site (`Pberghei.gff3.gz`).**

In [1]:
wget ftp://ftp.sanger.ac.uk/pub/project/pathogens/gff3/CURRENT/Pberghei.gff3.gz

--2018-12-11 10:49:33--  ftp://ftp.sanger.ac.uk/pub/project/pathogens/gff3/CURRENT/Pberghei.gff3.gz
           => ‘Pberghei.gff3.gz’
Resolving ftp.sanger.ac.uk (ftp.sanger.ac.uk)... 193.62.203.115
Connecting to ftp.sanger.ac.uk (ftp.sanger.ac.uk)|193.62.203.115|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/project/pathogens/gff3/CURRENT ... done.
==> SIZE Pberghei.gff3.gz ... 9151369
==> PASV ... done.    ==> RETR Pberghei.gff3.gz ... done.
Length: 9151369 (8.7M) (unauthoritative)

100%[======================================>] 9,151,369   8.71MB/s   in 1.0s   

2018-12-11 10:49:34 (8.71 MB/s) - ‘Pberghei.gff3.gz’ saved [9151369]



Now, earlier on, you will have jotted down the location of PBANKA_KO in the genome.

**Search for a gene with the same co-ordinates as PBANKA_KO in the reall annotation file.**

In [3]:
zgrep "PbANKA_09_v3.*gene.*196868.*201157" Pberghei.gff3.gz

PbANKA_09_v3	chado	gene	196868	201157	.	+	.	ID=PBANKA_0905900;Name=AP2-O;previous_systematic_id=PB000572.01.0,PB300102.00.0,PB000264.01.0,PBANKA_090590;synonym=ApiAP2


Looks like PBANKA_KO is really **PBANKA_0905900**, better known as **AP2-O**, in disguise!

Looking into the literature, you will find that AP2-O encodes a transcription factor and plays a role in ookinete development. Thus, it makes sense that knocking out this gene will result in differential expression of genes involved in cell structure, cycle and motility.

In [14]:
awk -F'\t' '$4 < 0.01 && $5 > 0  {OFS="\t"; print $1,$2,$4,$5}' kallisto.results | sort -t$'\t' -k4 -nr | head -25

PBANKA_1006400	conserved Plasmodium protein, unknown function	8.43217106880059e-37	6.13291805548218
PBANKA_1037800	secreted ookinete adhesive protein	2.93746634002303e-278	5.30222160021813
PBANKA_0800500	chitinase	3.45232774701234e-204	5.2711755125222
PBANKA_0201700	conserved Plasmodium protein, unknown function	4.18193490658365e-74	5.15186429597038
PBANKA_0609600	heat shock protein 20, putative	1.29020957359245e-19	4.97390138802336
PBANKA_1119200	secreted ookinete protein 25 //  ookinete surface-associated protein 8	1.51264472823999e-108	4.89652647824314
PBANKA_1106900	PIMMS2 protein	5.33282623432314e-193	4.48135406144008
PBANKA_1457700	conserved Plasmodium protein, unknown function	1.44669681853367e-81	3.84309630526164
PBANKA_1017000	CorA-like Mg2+ transporter protein, putative	1.07807943500591e-75	3.69705577143191
PBANKA_0410650	conserved Plasmodium protein, unknown function	1.64944349522743e-57	3.67381093875636
PBANKA_0412900	circumsporozoite- and TRAP-related protein	2.81639952956

In [13]:
awk -F'\t' '$4 < 0.01 && $5 < 0  {OFS="\t"; print $1,$2,$4,$5}' kallisto.results | sort -t$'\t' -k4 -nr | head -25

PBANKA_1438000	cAMP-dependent protein kinase regulatory subunit, putative	0.00903851762262946	-0.389994129261354
PBANKA_1459400	conserved Plasmodium protein, unknown function	0.00903782721656809	-0.391323453116088
PBANKA_1441300	RAP protein, putative	0.00858416310736202	-0.39340997629559
PBANKA_1016300	proliferation-associated protein 2g4, putative	0.0064420363837743	-0.400815377153955
PBANKA_1208000	tubulin--tyrosine ligase, putative	0.00687563554289438	-0.405613504921808
PBANKA_0307000	60S ribosomal protein L37ae, putative	0.0092366702981869	-0.407233797568315
PBANKA_1362400	conserved Plasmodium protein, unknown function	0.00817177618616866	-0.409404094365412
PBANKA_1320700	signal peptide peptidase, putative	0.00714889632345314	-0.411057367014384
PBANKA_1228200	glutamate dehydrogenase, putative	0.00502484463091534	-0.41300870229637
PBANKA_0602300	conserved Plasmodium protein, unknown function	0.00622054560962365	-0.413075585629026
PBANKA_1349700	conserved Plasmodium protein, unknown 